# Parsowanie html-i z bs4
## Przykłady
Dokumentacja jest dość przystępna:

https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [49]:
import bs4
import requests

### Pliki HTML
Składają się z treści tekstowej, w postaci znaczników pojedyńczych lub sparowanych, sparowane znaczniki mogą zawierać w swojej treści inne znaczniki i tekst:

- `<znacznik1>`

- `<znacznik2>` `some data` `</znacznik2>`
"
Albo bardziej złożony przykład:

`<znacznik1>
<znacznik2> COŚ </znacznik2>
<znacznik3>
    <znacznik31> 
        <znacznik311> 
        treść tekstowa
        </znacznik311>
        <znacznik312 atrybut="wartość"> 
    </znacznik31>
    <znacznik32>
        <a href="https://google.com"> Tekst linku </a>
    </znacznik32>
</znacznik3>  `  `

### Pobieranie strony .html i parsowanie bs4

In [50]:
URL="http://europepmc.org/journals/3245/"
req=requests.get(URL)
html_txt=req.text
bs4_html=bs4.BeautifulSoup(html_txt, 'html.parser') 

In [51]:
bs4_html

<!DOCTYPE html>

<html lang="en" prefix="dc: http://purl.org/dc/elements/1.1/#; dcterms: http://purl.org/dc/terms/#">
<head><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta charset="utf-8"/><meta content="width=device-width" name="viewport"/><script src="./wicket/resource/org.apache.wicket.resource.JQueryResourceReference/jquery/jquery-1.11.1.min-ver-8101D596B2B8FA35FE3A634EA342D7C3.js" type="text/javascript"></script>
<link href="/css_min/min/orcid-min.css?v=12087" rel="stylesheet" type="text/css"/>
<script src="./wicket/resource/org.europepmc.account.AccountSigninPanel/AccoutSigninPanel-ver-23536C62407C81CFCDCB80EBF689D4C9.js" type="text/javascript"></script>
<title>Europe PMC</title><link href="/css/chunk-00f8ff4e.a8744ce3.css" rel="prefetch"/><link href="/css/chunk-0174ae6e.dbe6b2e6.css" rel="prefetch"/><link href="/css/chunk-0e93dfce.dd3b2cb4.css" rel="prefetch"/><link href="/css/chunk-295e29ee.f8971f0e.css" rel="prefetch"/><link href="/css/chunk-2a369350.d9f0ba2d.css" 

### Znajdź pierwszy znacznik `<a>`w bs4_html

In [52]:
a_tag=bs4_html.find('a')
print(type(a_tag))     # klasa tag zawiera wszystkie info o znaczniku i jego treść
print(a_tag)
print(a_tag.attrs)       #lista atrybutów
print("href of a_tag is" +a_tag['href'])      # dostęp do atrybutu href
print(str(a_tag)) # string

<class 'bs4.element.Tag'>
<a class="orcid_signin_text" href="/accounts/login" id="login-bar--signin-link">Sign in</a>
{'id': 'login-bar--signin-link', 'class': ['orcid_signin_text'], 'href': '/accounts/login'}
href of a_tag is/accounts/login
<a class="orcid_signin_text" href="/accounts/login" id="login-bar--signin-link">Sign in</a>


### Znajdź wszystkie `<a>`w bs4_html

In [53]:
a_tagList=bs4_html.findAll('a') # lista wyników, obiektów klasy bs4 tag
print(a_tagList[0:3])
a_tag2= a_tagList[2]
print("Tag name: "+a_tag2.name)

[<a class="orcid_signin_text" href="/accounts/login" id="login-bar--signin-link">Sign in</a>, <a class="orcid_signin_text" href="/accounts/registration">Create an account</a>, <a accesskey="U" href="/" target="_top">
<span class="screenReader">Europe PMC</span>
</a>]
Tag name: a


### Znajdź znaczniki `<table>`

In [54]:
t_tags= bs4_html.findAll('table')
print(len(t_tags))

11


### Szukanie w danym znaczniku `<table>`

In [55]:
t_tag0 = t_tags[0]

tab0links=t_tag0.findAll('a')[:3]
tab0links

[<a class="navlink" href="/journals/">Journal List</a>,
 <a class="arc-issue" href="/issues/357923/">v.9(1): <span class="nowrap">1–41</span><br/>2022 Jan</a>,
 <a class="arc-issue" href="/issues/360683/">v.9(2): <span class="nowrap">42–84</span><br/>2022 Feb</a>]

### Szukanie w oparciu o flitr, wszystkie tagi danego rodzaju zawierające atrybut href

In [56]:
TrueLinks=bs4_html.findAll('a', href=True)

### Filtr oparty o funkcję
#### Zwracamy tagi lub tag tylko gdy wartość funkcji obliczonej na danym tagu to True

In [57]:
def filter_fun(tag):
    """ Zwraca True, jeżeli w treści zamkniętej w danym tagu <div> znajdzie się słowo Feb """
    return tag.name=='div' and 'Feb' in str(tag)
    

In [58]:
DivFebs=bs4_html.findAll(filter_fun)

In [59]:
print(len(DivFebs))

9


In [60]:
print(DivFebs[1]) # dużo...

<div id="epmc_pageBody">
<div class="articleContentPage">
<script src="/Scripts/recaptcha_ajax.js?v=12087" type="text/javascript"></script>
<div><div class="epmc-elements" id="app"><div data-v-7fb82372=""><div class="" data-v-7fb82372="" id="search-panel"><div class="grid-row" data-v-7fb82372=""><div class="col-16" data-v-7fb82372=""><h1 data-v-7fb82372=""><label class="semi-bold" data-v-7fb82372="" for="banner--search-input">Search life-sciences literature
<span class="regular" data-v-7fb82372="" id="article-count">(<a data-v-7fb82372="" href="/search?query=SRC:*" id="article-count-link">41,433,338</a>
articles, preprints and more)</span></label></h1><div data-v-7fb82372="" id="search-bar"><form action="/search" data-v-7fb82372="" id="search-bar-inputs" method="get"><input data-v-7fb82372="" id="banner--search-input" name="query" type="text"/><button class="regular" data-v-7fb82372="" id="banner--search-button" type="submit"><i class="fa fa-search" data-v-7fb82372=""></i><span data-v-

In [61]:
print(DivFebs[1].find(lambda x: 'Feb' in x.text and x.name=='a'))

<a class="arc-issue" href="/issues/360683/">v.9(2): <span class="nowrap">42–84</span><br/>2022 Feb</a>
